In [ ]:
# -------------- 1 ------------------

In [16]:
from pyspark.sql import SparkSession
from os import truncate
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [17]:
spark = SparkSession.builder.appName("CurrencyConversion").getOrCreate()

In [18]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/content/top_100_saas_companies_2025.csv")

In [19]:
df.show(5)

+------------+------------+--------------------+--------------------+-------------+------+---------+---------+--------------------+--------------------+---------+
|Company Name|Founded Year|                  HQ|            Industry|Total Funding|   ARR|Valuation|Employees|       Top Investors|             Product|G2 Rating|
+------------+------------+--------------------+--------------------+-------------+------+---------+---------+--------------------+--------------------+---------+
|   Microsoft|        1975|    Redmond, WA, USA| Enterprise Software|          $1B| $270B|      $3T|  221,000|Bill Gates, Paul ...|Azure, Office 365...|      4.4|
|  Salesforce|        1999|San Francisco, CA...|                 CRM|       $65.4M|$37.9B|  $227.8B|   75,000|Halsey Minor, Lar...|Sales Cloud, Serv...|      4.3|
|       Adobe|        1982|   San Jose, CA, USA|   Creative Software|        $2.5M|$19.4B|    $240B|   29,945|   Hambrecht & Quist|Creative Cloud, D...|      4.5|
|      Oracle|        

In [20]:
df.printSchema()

root
 |-- Company Name: string (nullable = true)
 |-- Founded Year: integer (nullable = true)
 |-- HQ: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Total Funding: string (nullable = true)
 |-- ARR: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Employees: string (nullable = true)
 |-- Top Investors: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- G2 Rating: double (nullable = true)



In [21]:
df = df.withColumn("splchars_removal", regexp_replace("Valuation",r"\s*\(.*\)",""))

In [22]:
def convert_currency(val):
    if val == "N/A":
        return 0
    val = val.replace("$", "").upper()
    if "T" in val:
        return float(val.replace("T", "")) * 1_000_000_000_000
    if "B" in val:
        return float(val.replace("B", "")) * 1_000_000_000
    if "M" in val:
        return float(val.replace("M", "")) * 1_000_000

In [23]:
currency_udf = udf(convert_currency, DoubleType())

In [24]:
df = df.withColumn("Funding_Num", currency_udf(col("Total Funding"))) \
       .withColumn("ARR_Num", currency_udf(col("ARR"))) \
       .withColumn("Valuation_Num", currency_udf(col("splchars_removal")))

In [25]:
df.select("Funding_Num", "ARR_Num", "Valuation_Num").show(truncate = False)

+--------------------+--------------------+-------------+
|Funding_Num         |ARR_Num             |Valuation_Num|
+--------------------+--------------------+-------------+
|1.0E9               |2.7E11              |3.0E12       |
|6.540000000000001E7 |3.79E10             |2.278E11     |
|2500000.0           |1.94E10             |2.4E11       |
|NULL                |5.29E10             |3.5E11       |
|NULL                |3.25E10             |2.15E11      |
|2.73E8              |1.44E10             |1.8E11       |
|8.25E7              |8.9E9               |1.47E11      |
|2.499E8             |7.3E9               |6.5E10       |
|1.455E8             |4.5E9               |8.5E10       |
|1.223E8             |7.1E9               |9.5E10       |
|6.0E7               |3.5E9               |5.5E10       |
|1.4E9               |2.8E9               |7.5E10       |
|1.005E8             |2.2E9               |3.2E10       |
|5.1429999999999994E8|2.5E9               |1.0E10       |
|1.4E9        

In [26]:
# ------------ 2 ---------------

In [27]:
from pyspark.sql.window import *

In [28]:
window_fun=Window.partitionBy("Industry").orderBy(col("Valuation_Num").desc())

In [29]:
df.withColumn("rank",rank().over(window_fun)).filter(col("rank")<=2).select("Industry","Company Name","valuation_num","rank").show()

+--------------------+------------+-------------+----+
|            Industry|Company Name|valuation_num|rank|
+--------------------+------------+-------------+----+
|                 APM| AppDynamics|        3.7E9|   1|
|                BNPL|      Affirm|       1.2E10|   1|
|Business Intellig...|      Looker|        2.6E9|   1|
|               CI/CD|    CircleCI|        1.7E9|   1|
|                 CRM|  Salesforce|     2.278E11|   1|
|        Card Issuing|     Marqeta|        4.3E9|   1|
|      Cloud Security|     Zscaler|       3.0E10|   1|
|      Cloud Security|    Netskope|        7.5E9|   2|
|       Cloud Storage|     Dropbox|        8.5E9|   1|
|       Cloud Storage|         Box|        3.5E9|   2|
|       Collaboration|        Miro|      1.75E10|   1|
|Collaboration Sof...|   Atlassian|       5.5E10|   1|
|      Communications|      Twilio|       1.2E10|   1|
|      Communications| RingCentral|        5.0E9|   2|
|        Construction|     Procore|        9.0E9|   1|
|      Con

In [30]:
# ----------- 3 -------------

In [36]:
df.show(5)

+------------+------------+--------------------+--------------------+-------------+------+---------+---------+--------------------+--------------------+---------+----------------+-------------------+-------+-------------+
|Company Name|Founded Year|                  HQ|            Industry|Total Funding|   ARR|Valuation|Employees|       Top Investors|             Product|G2 Rating|splchars_removal|        Funding_Num|ARR_Num|Valuation_Num|
+------------+------------+--------------------+--------------------+-------------+------+---------+---------+--------------------+--------------------+---------+----------------+-------------------+-------+-------------+
|   Microsoft|        1975|    Redmond, WA, USA| Enterprise Software|          $1B| $270B|      $3T|  221,000|Bill Gates, Paul ...|Azure, Office 365...|      4.4|             $3T|              1.0E9| 2.7E11|       3.0E12|
|  Salesforce|        1999|San Francisco, CA...|                 CRM|       $65.4M|$37.9B|  $227.8B|   75,000|Ha

In [39]:
window_fun = Window.partitionBy("Industry").orderBy(col("ARR_Num").desc())

In [40]:
df_with_gap = df.withColumn("Prev_ARR", lag("ARR_Num").over(window_fun)) \
                .withColumn("ARR_Difference", col("Prev_ARR") - col("ARR_Num"))

In [41]:
df_with_gap.select(
    "Industry",
    "Company Name",
    "ARR",
    "ARR_Num",
    "Prev_ARR",
    "ARR_Difference"
).orderBy("Industry", col("ARR_Num").desc()) \
 .show(truncate=False)

+----------------------+------------+------+--------------------+--------------------+--------------------+
|Industry              |Company Name|ARR   |ARR_Num             |Prev_ARR            |ARR_Difference      |
+----------------------+------------+------+--------------------+--------------------+--------------------+
|APM                   |AppDynamics |$600M |6.0E8               |NULL                |NULL                |
|BNPL                  |Affirm      |$1.6B |1.6E9               |NULL                |NULL                |
|Business Intelligence |Looker      |$150M |1.5E8               |NULL                |NULL                |
|CI/CD                 |CircleCI    |$100M |1.0E8               |NULL                |NULL                |
|CRM                   |Salesforce  |$37.9B|3.79E10             |NULL                |NULL                |
|Card Issuing          |Marqeta     |$717M |7.17E8              |NULL                |NULL                |
|Cloud Security        |Zsca

In [ ]:
# ------------ 4 --------------

In [32]:
rating_df= df.withColumn("rating_tier", when(col("G2 Rating") >= 4.7, "excellent").when((col("G2 Rating") >= 4.3) & (col("G2 Rating") < 4.7), "very good").when((col("G2 Rating") >= 4.0) & (col("G2 Rating") < 4.3), "good").otherwise("average"))
rating_df.select("Company Name", "G2 Rating", "rating_tier").show()

+------------+---------+-----------+
|Company Name|G2 Rating|rating_tier|
+------------+---------+-----------+
|   Microsoft|      4.4|  very good|
|  Salesforce|      4.3|  very good|
|       Adobe|      4.5|  very good|
|      Oracle|      4.0|       good|
|         SAP|      4.1|       good|
|      Intuit|      4.4|  very good|
|  ServiceNow|      4.4|  very good|
|     Workday|      4.2|       good|
|        Zoom|      4.5|  very good|
|     Shopify|      4.4|  very good|
|   Atlassian|      4.3|  very good|
|   Snowflake|      4.4|  very good|
|     HubSpot|      4.4|  very good|
|    DocuSign|      4.5|  very good|
|       Slack|      4.5|  very good|
|      Notion|      4.7|  excellent|
|     Datadog|      4.4|  very good|
|     MongoDB|      4.5|  very good|
|        Okta|      4.4|  very good|
|      Twilio|      4.3|  very good|
+------------+---------+-----------+
only showing top 20 rows



In [ ]:
# ----------------- 5 ----------------------

In [42]:
investor_tiers = spark.createDataFrame([
    ("Accel", "Tier 1"),
    ("Sequoia", "Tier 1"),
    ("Andreessen Horowitz", "Tier 1"),
    ("SoftBank", "Tier 2"),
    ("Lightspeed", "Tier 2"),
    ("Unknown", "Tier 3")
], ["Investor", "Tier"])

In [43]:
companies_first_investor=df.withColumn("f1_investor",split(col("Top Investors"),",")[0])

In [44]:
joining=companies_first_investor.join(investor_tiers,companies_first_investor.f1_investor==investor_tiers.Investor,how="left")

In [45]:
joining.select("Company Name","f1_investor","Tier","valuation").show()

+------------+-------------------+------+-------------------+
|Company Name|        f1_investor|  Tier|          valuation|
+------------+-------------------+------+-------------------+
|        Zoom|    Sequoia Capital|  NULL|               $85B|
|   Atlassian|     Accel Partners|  NULL|               $55B|
|       Slack|              Accel|Tier 1|$27.7B (Salesforce)|
|     Workday|  Greylock Partners|  NULL|               $65B|
|     Datadog|             ICONIQ|  NULL|               $44B|
|         SAP|       Dietmar Hopp|  NULL|              $215B|
|  ServiceNow|         JMI Equity|  NULL|              $147B|
|      Notion|     Index Ventures|  NULL|               $10B|
|     Shopify|           Bessemer|  NULL|               $95B|
|      Twilio|           Bessemer|  NULL|               $12B|
|      Intuit|    Sierra Ventures|  NULL|              $180B|
|   Snowflake|            Sequoia|Tier 1|               $75B|
|     MongoDB|            Sequoia|Tier 1|               $26B|
|      S

In [46]:
# --------------- 6 ----------------

In [47]:
industry_medians_df = spark.createDataFrame([
    ("Enterprise Software", 150_000_000_000),
    ("CRM", 100_000_000_000),
    ("AI", 70_000_000_000),
    ("HRTech", 50_000_000_000),
], ["Industry", "Median_Valuation"])

In [50]:
companies_with_median = df.join(industry_medians_df, on="Industry", how="left")

In [56]:
classified_companies = companies_with_median.withColumn(
    "Valuation_Position",
    when(col("Valuation") > col("Median_Valuation"), "Above Median")
    .otherwise("Below Median")
)

In [57]:
classified_companies.select("Company Name", "Industry", "Valuation", "Median_Valuation", "Valuation_Position").show(truncate=False)

+------------+----------------------+-------------------+----------------+------------------+
|Company Name|Industry              |Valuation          |Median_Valuation|Valuation_Position|
+------------+----------------------+-------------------+----------------+------------------+
|Zoom        |Video Communications  |$85B               |NULL            |Below Median      |
|Stripe      |Payments              |$65B               |NULL            |Below Median      |
|Atlassian   |Collaboration Software|$55B               |NULL            |Below Median      |
|Oracle      |Database & Enterprise |$350B              |NULL            |Below Median      |
|Workday     |HR & Finance          |$65B               |NULL            |Below Median      |
|Slack       |Team Communication    |$27.7B (Salesforce)|NULL            |Below Median      |
|Adobe       |Creative Software     |$240B              |NULL            |Below Median      |
|Microsoft   |Enterprise Software   |$3T                |150